In [28]:
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
#from Research import Research
import csv
import datetime
import gensim
from gensim import corpora
import string
from openpyxl import load_workbook
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from nltk import corpus
import re, numpy as np, pandas as pd
from pprint import pprint
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
#import Seaborn as sb

ModuleNotFoundError: No module named 'spacy'

## This is a software that uses LDA model from nltk library and creates a metric 


- How to run on Linux:
    - sudo pip install -U numpy
    - sudo pip install -U gensim
    - sudo pip install -U openpyxl
    - sudo pip install -U nltk
    - cd /usr/local/lib/python3.5/dist-packages
    - sudo python -m nltk.downloader stopwords
    - sudo python -m nltk.downloader wordnet
    
- Windows:
    - pip install -U nltk
    - pip install -U numpy
    - pip install -U gensim
    - pip install -U openpyxl
    - python
    <br>
    <font color=red>
    import nltk
    <br>
    nltk.download('stopwords')
    <br>
    nltk.download('wordnet')
    </font>

In [2]:
class Arion:
    @staticmethod
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    @staticmethod
    def weighted_mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs(((y_true*y_pred) - (y_pred*y_pred)) / (y_true* y_pred))) * 100

In [3]:
class Research:
    @staticmethod
    def parse_xls(xls_file=None):
        if not xls_file:
            return False
        df = pd.read_excel(xls_file)
        return list(df['Abstract Note'])

    @staticmethod
    def parse_scholar(query_word='security', size=6, initial_time=None, end_time=None):
        import scholarly
        q = '/scholar?lr=lang_us&q='+query_word+'&hl=en-US&as_vis=1&as_sdt=1,5'
        if(initial_time != None):
            q = q + '&as_ylo=' + initial_time
        if(end_time != None):
            q = q +'&as_yhi=' + end_time
        searchFilter = scholarly.search_pubs_custom_url(q)
        return [next(searchFilter).bib['abstract'] for i in range(size)]
    
    @staticmethod    
    def clean(doc, stopWords):
        with open(stopWords, "rb") as msw:
            meuStopWord = msw.read().decode('utf-8').split("\r\n")


        stop = set(stopwords.words('english'))
        exclude = set(string.punctuation)
        lemma = WordNetLemmatizer()
        
        stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
        punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
        return " ".join([i for i in normalized.split() if i not in meuStopWord])
    @staticmethod
    def save(path, doc_list):
        with open(path, "wb") as file:
            doc_str = str(doc_list).encode("utf-8")
            file.write(doc_str)

In [4]:
#Set variables
source = 'Papers.xlsx'
stopWords = 'BlockWords.txt'
clean_path = 'CleanArchive.txt'
log_path = 'BeforeArchive.txt'
topics = 10
words = 8

In [5]:
#prepare docs
#log = research.revoke(source)
doc_complete = Research.parse_xls(source)
doc_clean = [Research.clean(doc, stopWords).split() for doc in doc_complete]


In [6]:
#save this version and alterations
Research.save(log_path, []) #log)
Research.save(clean_path, doc_clean)

In [19]:
#dictionary and matrix
dictionary = corpora.Dictionary(doc_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [8]:
#lda model
LDA = gensim.models.ldamodel.LdaModel

In [9]:
#LDA result
result = LDA(doc_term_matrix, num_topics=topics, id2word=dictionary, passes=2)

## Metrics works

In [10]:
table = result.show_topics(formatted=False)

In [11]:
real = [0.01 for x in range(100)]

In [12]:
predict = []
for x in range(10):
    for y in range(10):
        predict.append(table[x][1][y][1])
predict[:6]

[0.022344183, 0.021724818, 0.021658245, 0.017381188, 0.01592131, 0.015065742]

In [13]:
print(Arion.mean_absolute_percentage_error(real,predict))
print(Arion.weighted_mean_absolute_percentage_error(real,predict))

32.99258541315794
32.99258555380613


# Graphic Runs

In [26]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

In [27]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(doc_clean, stop_words)  

NameError: name 'simple_preprocess' is not defined

In [21]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    
    sent_topics_df = pd.DataFrame()
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=result, corpus=doc_term_matrix, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)


NameError: name 'data' is not defined